# 13-L1 Problem 1

We return to the plane-strain compression problem where the goal is to predict von Mises stress at a node given a set of its features.

Now, you will look at ensemble methods in sklearn to determine how well they perform in this context.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor


def plot_shape(dataset, index, model=None, lims=None):
    x = dataset["coordinates"][index][:,0]
    y = dataset["coordinates"][index][:,1]

    if model is None:
        c = dataset["stress"][index]
    else:
        c = model.predict(dataset["features"][index])

    if lims is None:
        lims = [min(c),max(c)]    

    plt.scatter(x,y,s=5,c=c,cmap="jet",vmin=lims[0],vmax=lims[1])
    plt.colorbar(orientation="horizontal", shrink=.75, pad=0,ticks=lims)
    plt.axis("off")
    plt.axis("equal")

def plot_shape_comparison(dataset, index, model, title=""):
    plt.figure(figsize=[6,3.2], dpi=120)
    plt.subplot(1,2,1)
    plot_shape(dataset,index)
    plt.title("Ground Truth",fontsize=9,y=.96)
    plt.subplot(1,2,2)
    c = dataset["stress"][index]
    plot_shape(dataset, index, model, lims = [min(c), max(c)])
    plt.title("Prediction",fontsize=9,y=.96)
    plt.suptitle(title)
    plt.show()

def load_dataset(path):
    dataset = np.load(path)
    coordinates = []
    features = []
    stress = []
    N = np.max(dataset[:,0].astype(int)) + 1
    split = int(N*.8)
    for i in range(N):
        idx = dataset[:,0].astype(int) == i
        data = dataset[idx,:]
        coordinates.append(data[:,1:3])
        features.append(data[:,3:-1])
        stress.append(data[:,-1])
    dataset_train = dict(coordinates=coordinates[:split], features=features[:split], stress=stress[:split])
    dataset_test = dict(coordinates=coordinates[split:], features=features[split:], stress=stress[split:])
    X_train, X_test = np.concatenate(features[:split], axis=0), np.concatenate(features[split:], axis=0)
    y_train, y_test = np.concatenate(stress[:split], axis=0), np.concatenate(stress[split:], axis=0)
    return dataset_train, dataset_test, X_train, X_test, y_train, y_test

def get_shape(dataset,index):
    X = dataset["features"][index]
    y = dataset["stress"][index]
    return X, y

## Loading the data

First, complete the code below to load the data and plot the von Mises stress fields for a few shapes.  
You'll need to input the path of the data file, the rest is done for you. 

All training node features and outputs are in `X_train` and `y_train`, respectively. Testing nodes are in `X_test`, `y_test`.  

`dataset_train` and `dataset_test` contain more detailed information such as node coordinates, and they are separated by shape.  
Get features and outputs for a shape by calling `get_shape(dataset,index)`. `N_train` and `N_test` are the number of training and testing shapes in each of these datasets.

In [ ]:
# YOU MAY NEED TO EDIT data_path
data_path = "stress_nodal_features.npy"
dataset_train, dataset_test, X_train, X_test, y_train, y_test = load_dataset(data_path)
N_train = len(dataset_train["stress"])
N_test = len(dataset_test["stress"])

plt.figure(figsize=[15,3.2], dpi=150)
for i in range(5):
    plt.subplot(1,5,i+1)
    plot_shape(dataset_train,i)
    plt.title(f"Shape {i}")
plt.show()

## StackingRegressor

A StackingRegressor consists of $N$ fitted regression models, which it evaluates to make $N$ predictions. Then, these predictions are fed into another regression model, which makes a final prediction of the target.

### List of Regressors
First, initialize 4 regression models:
1. Linear Regression
2. Decision Tree regression, max depth 4
3. Decision Tree regression, max depth 8
4. Decision Tree regression, max depth 12

Then, store these in a list called `models`. 

In [ ]:
# YOUR CODE GOES HERE
# Define models, and put in a list called 'models'

named_models = [(f"Model {i+1}", model) for i, model in enumerate(models)]
print(*named_models, sep="\n")

### Final Regressor
Now make one more regressor, which will take as input the other four predictions, and combine them to make an improved prediction. 

This can be another linear regression model. Call it `final_model`.

In [ ]:
# YOUR CODE GOES HERE

### Creating and training the StackingRegressor

Finally, we can combine all of our models into a StackingRegressor model. We fit this just as we would fit any sklearn model. Because of the size of the dataset, this may take a few minutes.

In [ ]:
srm = StackingRegressor(named_models, final_model, verbose=True)
srm.fit(X_train, y_train)

## Performance of the model

Now we can investigate the performance of the model on test data, compared to constituent models. First, let's look at the performance of each individual model of our model `srm`. These can be accessed via `srm.estimators_`.

In [ ]:
for i, estimator in enumerate(srm.estimators_):
    print(f"\n{named_models[i][0]}:")
    train_err = mean_squared_error(y_train, estimator.predict(X_train))
    test_err = mean_squared_error(y_test, estimator.predict(X_test))
    print(f"Training MSE: {train_err:.2e}")
    print(f" Testing MSE: {test_err:.2e}")


### Stacking Regressor MSE on Test Data
Now compute the MSE of `srm` on training and testing data.

Note how the results, particularly on test data, compare to the individual models.

In [ ]:
# YOUR CODE GOES HERE